In [1]:
import nltk
from nltk.tag import BigramTagger
import pandas as pd
import json

import importlib
import nlp_pos
nlp_pos = importlib.reload(nlp_pos)

#nltk.download_gui()

# Read the review into a dataframe

In [2]:
reviews = []
with open('reviewSelected100.json','r') as json_file:
    for line in json_file:
        reviews.append(json.loads(line))

reviews_df = pd.DataFrame(reviews)

# Get 5 random reviews and store their indexes for the future

In [3]:
def get_random_indexes(num=5):
    random_rows = reviews_df.sample(n = num)
    return random_rows.index

review_indexes = [7077, 15123, 14929, 4627, 2000] #Retrieved with the above function
random_rows = reviews_df['text'].loc[review_indexes]
random_rows = random_rows.reset_index(drop=True)
random_rows.head()

0    Store is clean and toppings look yummy, BUT\n\...
1    I travel every week.  I stay in Hampton s a lo...
2    I came here to eat on a Saturday night and ord...
3    We used Truecar to find a car we kinda liked. ...
4    the service wasnt really good , it was dead sl...
Name: text, dtype: object

In [4]:
nlp_pos = importlib.reload(nlp_pos)
from nlp_pos import clean_text

for i in random_rows.index:
    random_rows[i] = clean_text(random_rows[i])
    print(i,random_rows[i])

0 store is clean and toppings look yummy but

the tart flavor taste so bad I have visited twice and both times it taste sour

please throw away bad yogurt
1 i travel every week i stay in hampton s a lot 
this one is not up to par the shuttle never arrived after an hour no one cared it is in a warehouse area and does not feel very safe 

so disappointed
2 i came here to eat on a saturday night and ordered the ayce ordered the spicy sushi roll and it did not look fresh the rice broke apart and salmon was unappealing later ordered some dimsum and it looked as if was microwaved i decided to give them the 3 star for their attentive service and willingness to move to a warm table since their original spot by the door was freezing
3 we used truecar to find a car we kind of liked ronnie texted me back right away this was early morning on a sunday she came in found out later it was her day off to show is the car she gave us the best service i have ever had while car shopping she talked with us 

In [5]:
import nltk
import nltk.tag as tag
import nlp_pos

train_corpus = nltk.corpus.treebank.tagged_sents()

tnt = tag.TnT()
tnt.train(train_corpus)

In [6]:
import nlp_pos
nlp_pos = importlib.reload(nlp_pos)

from nlp_pos import spacy_analyse_review,nltk_analyse_review
from nlp_pos import dict_of_sets_to_column,list_of_tuple_to_column

nlp_pos = importlib.reload(nlp_pos)

def analyse( analyse_method:callable, args:list, df_col_name:str="tags"):
    """
    returns:
    tag_dict_col_df: df of tags assigned to each unique word
    tagged_tk_df: df of each pos-tags of the text
    """
    tagged_tokens, tags_per_word = analyse_method(*args)
    tag_dict_col_df = dict_of_sets_to_column(tags_per_word,df_col_name)
    tagged_tk_df = list_of_tuple_to_column(tagged_tokens,df_col_name)
    return tag_dict_col_df, tagged_tk_df

def analyse_text(text:str):
    spacy_dict_df, spacy_pos_df = analyse(spacy_analyse_review,[text],"spacy")
    nltk_dict_df, nltk_pos_df = analyse(nltk_analyse_review,[text,None],"nltk")
    tnt_dict_df, tnt_pos_df = analyse(nltk_analyse_review,[text, tnt.tag],"tnt")

    tags_df = pd.concat([spacy_dict_df,nltk_dict_df['nltk'],tnt_dict_df['tnt']], axis=1)
    tagged_tk_df = pd.concat([spacy_pos_df,nltk_pos_df['nltk'], tnt_pos_df['tnt']], axis=1)

    return tags_df, tagged_tk_df

In [7]:
tags_per_word_df = []
tagged_tokens_df = []
for i in range(5):
    t1, t2 = analyse_text(random_rows[i])
    tags_per_word_df.append(t1)
    tagged_tokens_df.append(t2)


with pd.ExcelWriter('five_reviews_pos_tag.xlsx') as writer:
    for i in range(5):
        tagged_tokens_df[i].to_excel(writer, sheet_name=f'S{i}')

